<img src="1.png" width="700"><BR>

# 정의 : 주어진 문서에 대하여 각 문서에 어떤 주제들이 존재하는지에 대한 확률 모형

<img src="3.png" width="800"><BR>

<img src="2.png" width="700"><BR>

# 1. 데이터 로딩

실습 데이터는 위키피디아 만 개의 문장을 사용
위키 피디아 한국어 전체 다운로드 링크 : 
https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B2%A0%EC%9D%B4%EC%8A%A4_%EB%8B%A4%EC%9A%B4%EB%A1%9C%EB%93%9C


In [1]:
from collections import Counter
from gensim import corpora, models
from pprint import pprint
from functools import reduce
from konlpy.tag import Twitter #Komoran, Hannanum등 다른 형태소 분석기도 가능


In [2]:
import bokeh.sampledata
import numpy as np
from sklearn.manifold import TSNE

In [3]:
%%time
with open('/Users/song-yeongsug/anaconda/LDA0603/lda/kowiki.csv', 'r') as f:
     docs_wi = [line for line in f.read().splitlines()]


CPU times: user 7.03 ms, sys: 2.93 ms, total: 9.96 ms
Wall time: 14.2 ms


# 2. 자르기(Tokenize)

In [8]:
#-*- coding: utf-8 -*-
from konlpy.tag import Twitter; t = Twitter()
pos = lambda d: ['/'.join(p) for p in t.pos(d, stem=True, norm=True)]
texts_wi = [pos(doc) for doc in docs_wi]
print(texts_wi[0])

['\ufeff/Foreign', '!!/Punctuation', '늘다/Verb', '체스/Noun', '에서/Josa', "'/Punctuation", '훌륭하다/Adjective', '이동/Noun', "'/Punctuation", '을/Josa', '하다/Verb', '때/Noun', '를/Josa', '나타내다/Verb', './Punctuation']


In [13]:
dictionary = corpora.Dictionary(texts_wi)

In [14]:
[d for d in dictionary.items()]

[(0, '\ufeff/Foreign'),
 (1, '!!/Punctuation'),
 (2, '늘다/Verb'),
 (3, '체스/Noun'),
 (4, '에서/Josa'),
 (5, "'/Punctuation"),
 (6, '훌륭하다/Adjective'),
 (7, '이동/Noun'),
 (8, '을/Josa'),
 (9, '하다/Verb'),
 (10, '때/Noun'),
 (11, '를/Josa'),
 (12, '나타내다/Verb'),
 (13, './Punctuation'),
 (14, '수학/Noun'),
 (15, '이중/Noun'),
 (16, '팩토리얼/Noun'),
 (17, '!!!/Punctuation'),
 (18, '(/Punctuation'),
 (19, '칙칙/Adverb'),
 (20, '으로/Josa'),
 (21, '읽다/Verb'),
 (22, ')/Punctuation'),
 (23, '은/Noun'),
 (24, '1996/Number'),
 (25, '년/Noun'),
 (26, '결성/Noun'),
 (27, '되다/Verb'),
 (28, '미국/Noun'),
 (29, '의/Josa'),
 (30, '댄스/Noun'),
 (31, '펑크/Noun'),
 (32, '밴드/Noun'),
 (33, '이다/Josa'),
 (34, '"/Punctuation'),
 (35, 'Me/Alpha'),
 (36, 'and/Alpha'),
 (37, 'Giuliani/Alpha'),
 (38, 'Down/Alpha'),
 (39, 'by/Alpha'),
 (40, 'the/Alpha'),
 (41, 'Schoolyard/Alpha'),
 (42, 'a/Alpha'),
 (43, 'true/Alpha'),
 (44, 'story/Alpha'),
 (45, '나/Noun'),
 (46, '""/Punctuation'),
 (47, 'Must/Alpha'),
 (48, 'Be/Alpha'),
 (49, 'Moon/Alpha'),
 (

In [15]:
import re

In [17]:
corpora.Dictionary(texts_wi)

In [5]:


stop_words = ["이", "의","ㄴ","는", "것", "하", "님", "좀", "거", "시", "왜", "제", "게", "식", "수", "그", "더", "때", "또",  "저", "나", "안",
              "건", "요", "도", "왜", "어", "가", "를", "에", "것", "뿐"]



# 2.2 불용어를 처리하는 이유 :  많다는 것이 꼭 좋은 것은 아니다!

 (예) 세종코퍼스에서 가장 많은 수의 단어들을 보면  다음과 같다. 
 조사나 어미류는 어느 문서에나 많이 포함되어 있기 때문에 문서간 차이를 구별하기 어렵게 한다. 

<img src="kkma.png" width="500"><BR>

http://kkma.snu.ac.kr/search

# 3. 자른 토큰을 Integers로 인코딩

In [6]:
from gensim import corpora
dictionary_wi = corpora.Dictionary(texts_wi)
dictionary_wi.save('wi.dict')  # save dictionary to file for future use


# 4. TF-IDF(Term Frequency - Inverse Document Frequency) 계산

<img src="tfidf.png" width="600"><BR>

F(단어 빈도, term frequency)는 특정한 단어가 문서 내에 얼마나 자주 등장하는지를 나타내는 값으로, 이 값이 높을수록 문서에서 중요하다고 생각할 수 있다. 하지만 단어 자체가 문서군 내에서 자주 사용되는 경우, 이것은 그 단어가 흔하게 등장한다는 것을 의미한다. 이것을 DF(문서 빈도, document frequency)라고 하며, 이 값의 역수를 IDF(역문서 빈도, inverse document frequency)라고 한다. TF-IDF는 TF와 IDF를 곱한 값이다.


tf(t,d) = 0.5 + 0.5 x f(t,d) / (문서 내 단어들의 f(t,d) 값 중 최댓값) 


IDF 값은 문서군의 성격에 따라 결정된다. 예를 들어 '원자'라는 낱말은 일반적인 문서들 사이에서는 잘 나오지 않기 때문에 IDF 값이 높아지고 문서의 핵심어가 될 수 있지만, 원자에 대한 문서를 모아놓은 문서군의 경우 이 낱말은 상투어가 되어 각 문서들을 세분화하여 구분할 수 있는 다른 낱말들이 높은 가중치를 얻게 된다.

idf(t,D) = log ( 전체 문서의 갯수 / 단어 t가 포함된 문서의 수 )


In [7]:
from gensim import models
tf_wi = [dictionary_wi.doc2bow(text) for text in texts_wi]
tfidf_model_wi = models.TfidfModel(tf_wi)
tfidf_wi = tfidf_model_wi[tf_wi]
corpora.MmCorpus.serialize('wi.mm', tfidf_wi) # save corpus to file for future use

# print first 10 elements of first document's tf-idf vector
print(tfidf_wi.corpus[0][:10])
# print top 10 elements of first document's tf-idf vector
print(sorted(tfidf_wi.corpus[0], key=lambda x: x[1], reverse=True)[:10])
# print token of most frequent element
print(dictionary_wi.get(414))

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1)]
[(5, 2), (0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (6, 1), (7, 1), (8, 1), (9, 1)]
죽이다/Verb


# Train topic model  

1. LSI

In [8]:
ntopics, nwords = 3, 5
lsi_wi = models.lsimodel.LsiModel(tfidf_wi, id2word=dictionary_wi, num_topics=ntopics)
print(lsi_wi.print_topics(num_topics=ntopics, num_words=nwords))

[(0, '0.580*")/Punctuation" + 0.520*"(/Punctuation" + 0.215*"년/Noun" + 0.164*"遼/Foreign" + 0.162*"요/Noun"'), (1, '0.498*"=/Punctuation" + 0.490*"|/Punctuation" + 0.484*"align/Alpha" + 0.375*"right/Alpha" + 0.175*"center/Alpha"'), (2, '0.304*",/Punctuation" + 0.292*""/Punctuation" + 0.271*"일/Noun" + 0.255*"월/Noun" + 0.203*"./Punctuation"')]


# 2. LDA

In [9]:
import numpy as np; np.random.seed(42)  # optional
lda_wi = models.ldamodel.LdaModel(tfidf_wi, id2word=dictionary_wi, num_topics=ntopics)
print(lda_wi.print_topics(num_topics=ntopics, num_words=nwords))

[(0, '0.010*"(/Punctuation" + 0.009*")/Punctuation" + 0.008*"베트남/Noun" + 0.008*"왕조/Noun" + 0.008*"李朝/Foreign"'), (1, '0.035*")/Punctuation" + 0.033*"(/Punctuation" + 0.015*"년/Noun" + 0.013*"북송/Noun" + 0.013*"北宋/Foreign"'), (2, '0.013*")/Punctuation" + 0.012*"(/Punctuation" + 0.009*"()/Punctuation" + 0.008*"日本/Foreign" + 0.008*"고려/Noun"')]


# Scoring documents

In [10]:
bow = tfidf_model_wi[dictionary_wi.doc2bow(texts_wi[0])]
sorted(lsi_wi[bow], key=lambda x: x[1], reverse=True)
sorted(lda_wi[bow], key=lambda x: x[1], reverse=True)


[(2, 0.82852782399405878),
 (1, 0.086190671003169864),
 (0, 0.085281505002771346)]

In [11]:
bow = tfidf_model_wi[dictionary_wi.doc2bow(texts_wi[1])]
sorted(lsi_wi[bow], key=lambda x: x[1], reverse=True)
sorted(lda_wi[bow], key=lambda x: x[1], reverse=True)


[(2, 0.77378349758514831), (1, 0.11931053162638963), (0, 0.10690597078846202)]

# word2vec

# 1.데이터 로드 

In [12]:

%%time
with open('/Users/song-yeongsug/anaconda/LDA0603/lda/kowiki.csv', 'r') as f:
    docs_wi = [line for line in f.read().splitlines()]

CPU times: user 7.41 ms, sys: 2.1 ms, total: 9.51 ms
Wall time: 8.39 ms


# 2. 토큰으로 자르기

In [13]:
from konlpy.tag import Twitter; t = Twitter()
pos = lambda d: ['/'.join(p) for p in t.pos(d)]
texts_wi = [pos(doc) for doc in docs_wi]

# 3. Train

In [14]:
from gensim.models import word2vec
wv_model_wi = word2vec.Word2Vec(texts_wi)
wv_model_wi.init_sims(replace=True)
wv_model_wi.save('wi_word2vec.model')

# 4. Test

In [16]:

wv_model_wi.most_similar(pos('영국'))

[('전/Noun', 0.9985165596008301),
 ('독립운동가/Noun', 0.9980098009109497),
 ('1957/Number', 0.9978177547454834),
 ('정치인/Noun', 0.9977759122848511),
 ('아나운서/Noun', 0.9975315928459167),
 ('물리학자/Noun', 0.9974913597106934),
 ('브라질/Noun', 0.9969936609268188),
 ('잉글랜드/Noun', 0.9969726800918579),
 ('프랑스/Noun', 0.9968726634979248),
 ('희극인/Noun', 0.9968183040618896)]

In [17]:
wv_model_wi.most_similar(pos('월'))

[('일/Noun', 0.9949101209640503),
 ('10/Number', 0.9877444505691528),
 ('일과/Noun', 0.9798873066902161),
 ('12/Number', 0.9798750877380371),
 ('9/Number', 0.9661765098571777),
 ('그레고리력/Noun', 0.9636373519897461),
 ('율리우스력/Noun', 0.9635851383209229),
 ('5/Number', 0.9625630378723145),
 ('11/Number', 0.9582865238189697),
 ('1/Number', 0.9512710571289062)]

# 시각화

In [18]:

top = 1000 #top은 표시할 단어의 수

vectors = wv_model_wi.wv.syn0[:top]
labels = wv_model_wi.wv.index2word[:top] 

from sklearn.manifold import TSNE


if np.shape(vectors)[1] > 2:
    tsne = TSNE(perplexity=10, n_components=2, init='random', n_iter=1000, verbose=1, learning_rate=5000, method='exact')
    vectors = tsne.fit_transform(vectors)


#source = ColumnDataSource(data=dict(x=vectors.T[0], y=vectors.T[1], word=labels))
    




[t-SNE] Computing pairwise distances...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 0.022599
[t-SNE] KL divergence after 100 iterations with early exaggeration: 19.624348
[t-SNE] Error after 175 iterations: 19.624348


In [19]:

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, LabelSet

In [20]:
source = ColumnDataSource(data=dict(x=vectors.T[0], y=vectors.T[1], word=labels))

In [21]:
label_set = LabelSet(x='x',y='y',text='word',source=source)

In [22]:
tools = "pan,wheel_zoom,box_zoom,reset,resize"

In [23]:
p = figure(plot_width=900,plot_height=900,tools=[tools],title='title')

In [24]:

p.circle('x','y',size=5,source=source,alpha=0.6)

GlyphRenderer(id='54cf5f82-12ea-433b-a635-4c99816e0ba0', ...)

In [25]:
p.add_layout(label_set)

<img src="space.png" width="700"><BR>

In [26]:
show(p)

# Working in the Notebook

http://bokeh.pydata.org/en/latest/docs/user_guide/notebook.html

In [35]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row

In [36]:
output_notebook()

Loading BokehJS ...

감사합니다.